# **Spectral Index Evaluation of Land Use Compliance with the PBOT in La Ceja**
This project aims to analyze and quantify the urban expansion in La Ceja, a municipality located in Eastern Antioquia (Colombia), between the years 2018 and 2025. Using Sentinel-2 multispectral satellite imagery, we calculate spectral indices such as NDVI (Normalized Difference Vegetation Index) and NDBI (Normalized Difference Built-up Index) to detect changes in vegetation cover and built-up areas.

These spatial transformations are then compared with the urban growth zones defined in the Municipal Land Use Plan (PBOT) established by Agreement 001 of 2018. The overlay between remote sensing analysis and official planning boundaries allows us to assess whether recent urban development has occurred within legally designated areas or has exceeded regulated expansion zones.

This approach provides technical evidence to support sustainable territorial management, monitor compliance with planning regulations, and inform future decision-making by local authorities. The project also explores the effectiveness of remote sensing as a tool for land-use monitoring in rapidly growing intermediate municipalities.


In [ ]:
import os
import rasterio
from rasterio.mask import mask
from pyproj import Transformer

folder_b4 = "carpeta_b4"  # Carpeta con las imágenes B4 originales
folder_b5 = "carpeta_b5"  # Carpeta con las imágenes B5 originales

output_b4 = "recortadas_b4"  # Carpeta de salida para B4
output_b5 = "recortadas_b5"  # Carpeta de salida para B5

os.makedirs(output_b4, exist_ok=True)
os.makedirs(output_b5, exist_ok=True)

# Coordenadas (lon, lat) - WGS84
coords_wgs84 = [
    (6.015422, -75.458237),  # inf izq
    (6.015422, -75.407993),  # inf der
    (6.040884, -75.407993),  # sup der
    (6.040884, -75.458237),  # sup izq
    (6.015422, -75.458237)   # cierre
]

# === TRANSFORMACIÓN A UTM ZONA 10N ===
transformer = Transformer.from_crs("EPSG:4326", "EPSG:32610", always_xy=True)
coords_utm = [transformer.transform(lon, lat) for lon, lat in coords_wgs84]

polygon = {
    "type": "Polygon",
    "coordinates": [coords_utm]
}

# === FUNCIONES ===
def recortar_guardar(input_path, output_path, polygon):
    with rasterio.open(input_path) as src:
        out_image, out_transform = mask(src, [polygon], crop=True)
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })
        with rasterio.open(output_path, "w", **out_meta) as dest:
            dest.write(out_image)

# === PROCESAR TODAS LAS IMÁGENES ===
for archivo_b4 in os.listdir(folder_b4):
    if archivo_b4.endswith(".TIF"):
        ruta_b4 = os.path.join(folder_b4, archivo_b4)

        # Buscar archivo B5 correspondiente (por nombre)
        archivo_b5 = archivo_b4.replace("B4", "B5")
        ruta_b5 = os.path.join(folder_b5, archivo_b5)

        if os.path.exists(ruta_b5):
            print(f"✅ Procesando: {archivo_b4} y {archivo_b5}")

            salida_b4 = os.path.join(output_b4, f"rec_{archivo_b4}")
            salida_b5 = os.path.join(output_b5, f"rec_{archivo_b5}")

            recortar_guardar(ruta_b4, salida_b4, polygon)
            recortar_guardar(ruta_b5, salida_b5, polygon)
        else:
            print(f"❌ No se encontró B5 para: {archivo_b4}")


FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: 'carpeta_b4'